## Import packages

In [1]:
import random
import itertools
import numpy as np
from dataclasses import dataclass

## Configuration for the subQUBO Algorithm

In [2]:
N_I = 20 # インスタンスプールの解の数
N_E = 10 # subQUBOを抽出する回数
N_S = 10 # N_I個のインスタンスプールからランダムに抽出する解の数
sub_qubo_size = 5 # subQUBOのスピン数

## Make QUBO

In [3]:
# 簡単化のため，TSPで考える
NUM_CITY = 4
ALPHA = 1
np.random.seed(0)
num_spin = NUM_CITY ** 2

distance_mtx = np.random.randint(1, 100, (NUM_CITY, NUM_CITY))
distance_mtx = np.tril(distance_mtx) + np.tril(distance_mtx).T - 2 * np.diag(distance_mtx.diagonal())

# <<< Objective term >>>
qubo_obj = np.zeros((NUM_CITY**2, NUM_CITY**2), dtype=np.int32)
for t_u_v in itertools.product(range(NUM_CITY), repeat=3):
    t, u, v = t_u_v[0], t_u_v[1], t_u_v[2]
    idx_i = NUM_CITY * t + u
    if t < NUM_CITY - 1:
        idx_j = NUM_CITY * (t + 1) + v
    elif t == NUM_CITY - 1:
        idx_j = v
    qubo_obj[idx_i, idx_j] += distance_mtx[u, v]
qubo_obj = np.triu(qubo_obj) + np.tril(qubo_obj).T - np.diag(np.diag(qubo_obj))

# <<< Constraint term >>>
qubo_constraint = np.zeros((NUM_CITY**2, NUM_CITY**2), dtype=np.int32)
# Calculate constraint term1 : 1-hot of horizontal line
for t in range(NUM_CITY):
    for u in range(NUM_CITY - 1):
        for v in range(u + 1, NUM_CITY):
            qubo_constraint[NUM_CITY*t+u, NUM_CITY*t+v] += ALPHA * 2
# Linear term
for t_u in itertools.product(range(NUM_CITY), repeat=2):
    qubo_constraint[NUM_CITY*t_u[0]+t_u[1], NUM_CITY*t_u[0]+t_u[1]] += ALPHA * (-1)
const_constraint = ALPHA * NUM_CITY

# Calculate constraint term2 : 1-hot of vertical line
# Quadratic term
for u in range(NUM_CITY):
    for t1 in range(NUM_CITY - 1):
        for t2 in range(t1+1, NUM_CITY):
            qubo_constraint[NUM_CITY*t1+u, NUM_CITY*t2+u] += ALPHA * 2
# Linear term
for u_t in itertools.product(range(NUM_CITY), repeat=2):
    qubo_constraint[NUM_CITY*u_t[1]+u_t[0], NUM_CITY*u_t[1]+u_t[0]] += ALPHA * (-1)
const_constraint += ALPHA * NUM_CITY

qubo_obj, qubo_constraint

(array([[ 0,  0,  0,  0,  0, 68, 37, 89,  0,  0,  0,  0,  0, 68, 37, 89],
        [ 0,  0,  0,  0, 68,  0, 88, 13,  0,  0,  0,  0, 68,  0, 88, 13],
        [ 0,  0,  0,  0, 37, 88,  0, 59,  0,  0,  0,  0, 37, 88,  0, 59],
        [ 0,  0,  0,  0, 89, 13, 59,  0,  0,  0,  0,  0, 89, 13, 59,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 68, 37, 89,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 68,  0, 88, 13,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 37, 88,  0, 59,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 89, 13, 59,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 68, 37, 89],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 68,  0, 88, 13],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 37, 88,  0, 59],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 89, 13, 59,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  

## Prepare the dataclass for Instance Pool

In [4]:
@dataclass
class Solution():
    """
    Solution information.

    Attributes:
        x (np.ndarray): n-sized solution composed of binary variables
        energy_all (float): energy value obtained from QUBO-matrix of all term
        energy_obj (float): energy value obtained from QUBO-matrix of objective term
        energy_constraint (float): energy value obtained from QUBO-matrix of constraint term
        constraint (bool): flag whether the solution satisfies the given constraint
    """
    x: np.ndarray
    energy_all: float = 0
    energy_obj: float = 0
    energy_constraint: float = 0
    constraint: bool = True

    @classmethod
    def energy(cls, qubo:np.ndarray, x: np.ndarray, const=0) -> float:
        """
        Calculate the enrgy from the QUBO-matrix & solution x

        Args:
            qubo (np.ndarray): n-by-n QUBO-matrix
            x (np.ndarray): n-sized solution composed of binary variables
            const (int, optional): _description_. Defaults to 0.

        Returns:
            float: Energy value.
        """
        return float(np.dot(np.dot(x, qubo), x) + const)

    @classmethod
    def check_constraint(cls, qubo: np.ndarray, x: np.ndarray, const=0) -> bool:
        """
        Check whether the solution satisfies the constraints.

        Args:
            qubo (np.ndarray): QUBO-model of the constraint term.
            x (np.ndarray): solution that you want to check.
            const (int, optional): constant of the constraint term. Defaults to 0.

        Returns:
            bool: Return True if the solution satisfy.
                  Return False otherwise.
        """
        return True if cls.energy(qubo, x, const) == 0 else False


## subQUBO Hybrid Annealing Algorithm

In [5]:
# https://ieeexplore.ieee.org/document/9664360

# <<< Line 2-4 >>>
# Initialize the Instance Pool
pool = []
for i in range(N_I):
    # ====================
    # 実験時にはここを変更する
    x = np.random.randint(0, 2, num_spin) # 今は一旦ランダムに解を生成
    # ====================
    energy_obj = Solution.energy(qubo_obj, x)
    energy_constraint = Solution.energy(qubo=qubo_constraint, x=x, const=const_constraint)
    pool.append(
        Solution(
            x = x,
            energy_all = energy_obj + energy_constraint,
            energy_obj = energy_obj,
            energy_constraint = energy_constraint,
            constraint = Solution.check_constraint(qubo=qubo_constraint, x=x, const=const_constraint)
        )
    )
ascending_order_idx = np.argsort(np.array(list(map(lambda sol: sol.energy_all, pool))))
pool = [pool[i] for i in ascending_order_idx]

# <<< Line 5 >>>
# Find the best solution
ascending_order_idx = np.argsort(np.array(list(map(lambda sol: sol.energy_all, pool))))
x_best = pool[ascending_order_idx[0]]

for _ in range(1): # <<< Line 6 >>>
    # <<< Line 7-8 >>>
    # Obtain a quasi-ground-state solution for every N_I solution instance by a classical QUBO solver
    for solution_i in pool:
        # ====================
        # 実験時にはここを変更する
        x = np.random.randint(0, 2, num_spin) # 今は一旦ランダムに解を生成
        # ====================

        # Update the solution info
        solution_i.x = x
        energy_obj = solution_i.energy(qubo_obj, x)
        energy_constraint = solution_i.energy(qubo_constraint, x, const_constraint)
        solution_i.energy_all = energy_obj + energy_constraint
        solution_i.energy_obj = energy_obj
        solution_i.energy_constraint = energy_constraint
        solution_i.constraint = solution_i.check_constraint(qubo=qubo_constraint, x=x, const=const_constraint)

    for i in range(N_E): # <<< Line 9 >>>
        # <<< Line 10 >>>
        # Select N_S solution instance randomly from the pool
        n_s_pool = random.sample(pool, N_S)

        # <<< Line 11-14 >>>
        # Calculate variance of each spin x_i in N_S instance poolSolution.check_constraint(qubo_constraint, x, const_constraint)
        vars_of_x = np.array([sum(n_s_pool[k].x[j] for k in range(N_S)) - N_S/2 for j in range(num_spin)])

        # <<< Line 15 >>>
        # Select a solution randomly from N_S solution instance pool as a tentative solution
        solution_tmp = random.choice(n_s_pool)

        # Extract a subQUBO
        extracted_spin_idx = np.argsort(vars_of_x)[:sub_qubo_size]
        non_extracted_spin_idx = np.argsort(vars_of_x)[sub_qubo_size:]
        subqubo_obj = np.array([[qubo_obj[j, k] for k in extracted_spin_idx] for j in extracted_spin_idx])
        subqubo_constraint = np.array([[qubo_constraint[j, k] for k in extracted_spin_idx] for j in extracted_spin_idx])
        for idx_i in range(sub_qubo_size):
            subqubo_obj[idx_i, idx_i] += sum(qubo_obj[idx_i, idx_j] * solution_tmp.x[idx_j] for idx_j in non_extracted_spin_idx)
            subqubo_constraint[idx_i, idx_i] += sum(qubo_constraint[idx_i, idx_j] * solution_tmp.x[idx_j] for idx_j in non_extracted_spin_idx)

        # <<< Line 16 >>>
        # Optimize the subQUBO using an Ising machine
        # ====================
        # 実験時にはここを変更する
        x_sub = np.random.randint(0, 2, sub_qubo_size) # 今は一旦ランダムに解を生成
        # ====================
        # Combine the quasi-ground-state solution from the subQUBO with the tentative solution X_t(solution_tmp)
        for idx, val in enumerate(extracted_spin_idx):
            solution_tmp.x[idx] = x_sub[idx]

        # <<< Line 17 >>>
        # Add the solution into the pool
        pool.append(solution_tmp)

    # <<< Line 18 >>>
    # Find the best soliution
    ascending_order_idx = np.argsort(np.array(list(map(lambda sol: sol.energy_all, pool))))
    x_best = pool[ascending_order_idx[0]]

    # <<< Line 19 >>>
    # Arrange the N_I instance pool
    sorted_pool = [pool[i] for i in ascending_order_idx]
    pool = sorted_pool[:N_I]

pool, x_best

([Solution(x=array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), energy_all=158.0, energy_obj=156.0, energy_constraint=2.0, constraint=False),
  Solution(x=array([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]), energy_all=265.0, energy_obj=259.0, energy_constraint=6.0, constraint=False),
  Solution(x=array([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]), energy_all=265.0, energy_obj=259.0, energy_constraint=6.0, constraint=False),
  Solution(x=array([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]), energy_all=316.0, energy_obj=306.0, energy_constraint=10.0, constraint=False),
  Solution(x=array([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]), energy_all=316.0, energy_obj=306.0, energy_constraint=10.0, constraint=False),
  Solution(x=array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]), energy_all=345.0, energy_obj=341.0, energy_constraint=4.0, constraint=False),
  Solution(x=array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0]), energy_all=378.0, energy_obj=370.0, ener